In [1]:
"""
Sistem de Testare Autovehicule - Simulare până la Defect sau Timp Planificat
Testează sisteme ale vehiculelor și monitorizează apariția defectelor
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from enum import Enum
import json
from pathlib import Path

In [2]:
# Configurare stiluri
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)


class SystemType(Enum):
    """Tipuri de sisteme auto"""
    ENGINE = "Motor"
    TRANSMISSION = "Transmisie"
    BRAKES = "Frâne"
    SUSPENSION = "Suspensie"
    ELECTRICAL = "Sistem Electric"
    COOLING = "Răcire"
    FUEL = "Alimentare Combustibil"
    EXHAUST = "Evacuare"
    STEERING = "Direcție"
    AIRBAGS = "Airbag-uri"


class DefectSeverity(Enum):
    """Severitatea defectelor"""
    MINOR = "Minoră"
    MODERATE = "Moderată"
    MAJOR = "Majoră"
    CRITICAL = "Critică"


@dataclass
class Defect:
    """Clasă pentru reprezentarea unui defect"""
    system: SystemType
    severity: DefectSeverity
    description: str
    time_occurred: float  # în ore
    odometer: float  # în km
    component: str
    error_code: Optional[str] = None


@dataclass
class TestResult:
    """Rezultat test pentru un sistem"""
    system: SystemType
    test_duration: float  # ore
    distance_covered: float  # km
    defects_found: List[Defect] = field(default_factory=list)
    passed: bool = True
    final_condition: float = 100.0  # procent stare


class VehicleSystem:
    """Clasă pentru reprezentarea unui sistem al vehiculului"""
    
    def __init__(self, system_type: SystemType, initial_condition: float = 100.0):
        self.system_type = system_type
        self.condition = initial_condition  # 0-100%
        self.total_hours = 0.0
        self.total_km = 0.0
        self.defects = []
        
        # Parametri specifici sistemului
        self._set_system_parameters()
    
    def _set_system_parameters(self):
        """Setează parametrii specifici pentru fiecare tip de sistem"""
        
        # Rata de uzură (procent/100 ore), probabilitate defect, componente
        params = {
            SystemType.ENGINE: {
                'wear_rate': 0.5,
                'failure_prob': 0.02,
                'components': ['Pistoane', 'Segmenți', 'Supape', 'Arbore cotit', 'Chiuloasă'],
                'mtbf': 5000  # Mean Time Between Failures (ore)
            },
            SystemType.TRANSMISSION: {
                'wear_rate': 0.4,
                'failure_prob': 0.03,
                'components': ['Ambreiaj', 'Cutie viteze', 'Diferențial', 'Planetare'],
                'mtbf': 4000
            },
            SystemType.BRAKES: {
                'wear_rate': 0.8,
                'failure_prob': 0.04,
                'components': ['Plăcuțe', 'Discuri', 'Pompă', 'Etriere', 'Lichid'],
                'mtbf': 2000
            },
            SystemType.SUSPENSION: {
                'wear_rate': 0.6,
                'failure_prob': 0.03,
                'components': ['Amortizoare', 'Arcuri', 'Bielete', 'Bucșe', 'Bară stabilizatoare'],
                'mtbf': 3000
            },
            SystemType.ELECTRICAL: {
                'wear_rate': 0.3,
                'failure_prob': 0.05,
                'components': ['Baterie', 'Alternator', 'Electromotor', 'Cablaj', 'Siguranțe'],
                'mtbf': 6000
            },
            SystemType.COOLING: {
                'wear_rate': 0.4,
                'failure_prob': 0.03,
                'components': ['Radiator', 'Pompă apă', 'Termostat', 'Ventilator', 'Furtunuri'],
                'mtbf': 4500
            },
            SystemType.FUEL: {
                'wear_rate': 0.3,
                'failure_prob': 0.02,
                'components': ['Pompă', 'Injectoare', 'Filtru', 'Rezervor', 'Senzori'],
                'mtbf': 5500
            },
            SystemType.EXHAUST: {
                'wear_rate': 0.7,
                'failure_prob': 0.04,
                'components': ['Catalizator', 'Filtru particule', 'Evacuare', 'Senzor oxigen'],
                'mtbf': 2500
            },
            SystemType.STEERING: {
                'wear_rate': 0.5,
                'failure_prob': 0.02,
                'components': ['Pompă servodirecție', 'Cremalieră', 'Timonerie', 'Coloană'],
                'mtbf': 4500
            },
            SystemType.AIRBAGS: {
                'wear_rate': 0.1,
                'failure_prob': 0.01,
                'components': ['Airbag șofer', 'Airbag pasager', 'Airbag lateral', 'Senzori'],
                'mtbf': 10000
            }
        }
        
        system_params = params.get(self.system_type, {
            'wear_rate': 0.5,
            'failure_prob': 0.03,
            'components': ['Component generic'],
            'mtbf': 4000
        })
        
        self.wear_rate = system_params['wear_rate']
        self.failure_prob = system_params['failure_prob']
        self.components = system_params['components']
        self.mtbf = system_params['mtbf']
    
    def simulate_operation(self, hours: float, avg_speed: float = 60.0):
        """
        Simulează operarea sistemului pentru un număr de ore
        
        Parameters:
        -----------
        hours : float
            Numărul de ore de operare
        avg_speed : float
            Viteza medie în km/h
        
        Returns:
        --------
        List[Defect] : Lista defectelor apărute în această perioadă
        """
        new_defects = []
        distance = hours * avg_speed
        
        # Uzură progresivă
        wear = self.wear_rate * (hours / 100)
        
        # Factori de stres suplimentari
        stress_factor = 1.0
        if self.condition < 50:
            stress_factor = 1.5
        elif self.condition < 30:
            stress_factor = 2.0
        
        self.condition -= wear * stress_factor
        self.condition = max(0, self.condition)
        
        # Verificare apariție defecte bazată pe MTBF și starea curentă
        # Probabilitate crește cu uzura
        adjusted_prob = self.failure_prob * (1 + (100 - self.condition) / 100)
        
        # Verificare defect la fiecare interval de timp
        time_steps = max(1, int(hours * 10))  # 10 verificări pe oră
        for step in range(time_steps):
            current_time = self.total_hours + (step / 10)
            current_km = self.total_km + (distance * step / time_steps)
            
            if np.random.random() < adjusted_prob / time_steps:
                # Apare un defect
                defect = self._generate_defect(current_time, current_km)
                new_defects.append(defect)
                self.defects.append(defect)
                
                # Defectele reduc starea sistemului
                severity_impact = {
                    DefectSeverity.MINOR: 5,
                    DefectSeverity.MODERATE: 15,
                    DefectSeverity.MAJOR: 30,
                    DefectSeverity.CRITICAL: 50
                }
                self.condition -= severity_impact[defect.severity]
                self.condition = max(0, self.condition)
        
        self.total_hours += hours
        self.total_km += distance
        
        return new_defects
    
    def _generate_defect(self, time: float, km: float) -> Defect:
        """Generează un defect aleator pentru sistem"""
        
        # Probabilități de severitate bazate pe starea sistemului
        if self.condition > 70:
            severities = [DefectSeverity.MINOR] * 7 + [DefectSeverity.MODERATE] * 3
        elif self.condition > 40:
            severities = [DefectSeverity.MINOR] * 4 + [DefectSeverity.MODERATE] * 4 + [DefectSeverity.MAJOR] * 2
        elif self.condition > 20:
            severities = [DefectSeverity.MODERATE] * 5 + [DefectSeverity.MAJOR] * 4 + [DefectSeverity.CRITICAL] * 1
        else:
            severities = [DefectSeverity.MAJOR] * 5 + [DefectSeverity.CRITICAL] * 5
        
        severity = np.random.choice(severities)
        component = np.random.choice(self.components)
        
        # Descrieri realiste pentru defecte
        descriptions = {
            (SystemType.ENGINE, DefectSeverity.MINOR): [
                f"Uzură ușoară la {component}",
                f"Vibrații minore la {component}",
                f"Pierdere ușoară de putere"
            ],
            (SystemType.ENGINE, DefectSeverity.CRITICAL): [
                f"Gripare {component}",
                f"Fisură în {component}",
                f"Pierdere completă de compresie"
            ],
            (SystemType.BRAKES, DefectSeverity.MAJOR): [
                f"Uzură avansată {component}",
                f"Pierdere parțială de eficiență frânare",
                f"Scurgere lichid de frână"
            ],
            (SystemType.ELECTRICAL, DefectSeverity.CRITICAL): [
                f"Scurtcircuit în {component}",
                f"Cădere completă {component}",
                f"Incendiu la {component}"
            ]
        }
        
        # Descriere generică dacă nu există una specifică
        desc_key = (self.system_type, severity)
        if desc_key in descriptions:
            description = np.random.choice(descriptions[desc_key])
        else:
            description = f"Defect {severity.value.lower()} la {component}"
        
        # Cod eroare (OBD-II style)
        error_code = f"P{np.random.randint(100, 999)}{np.random.randint(0, 9)}"
        
        return Defect(
            system=self.system_type,
            severity=severity,
            description=description,
            time_occurred=time,
            odometer=km,
            component=component,
            error_code=error_code
        )


class VehicleTester:
    """Clasă principală pentru testarea vehiculului"""
    
    def __init__(self, vehicle_id: str, systems: List[SystemType] = None):
        self.vehicle_id = vehicle_id
        self.test_start = datetime.now()
        
        # Inițializare sisteme
        if systems is None:
            systems = list(SystemType)
        
        self.systems = {
            sys_type: VehicleSystem(sys_type)
            for sys_type in systems
        }
        
        self.test_log = []
        self.total_test_time = 0.0
        self.total_distance = 0.0
    
    def run_test(self, 
                 planned_time: float = 100.0,
                 check_interval: float = 1.0,
                 avg_speed: float = 60.0,
                 stop_on_critical: bool = True,
                 stop_condition_threshold: float = 20.0) -> Dict:
        """
        Rulează testul pentru vehicul
        
        Parameters:
        -----------
        planned_time : float
            Timpul planificat de testare (ore)
        check_interval : float
            Intervalul de verificare (ore)
        avg_speed : float
            Viteza medie de testare (km/h)
        stop_on_critical : bool
            Oprește testul la defect critic
        stop_condition_threshold : float
            Oprește dacă starea unui sistem scade sub acest prag
        
        Returns:
        --------
        Dict : Rezultatele testului
        """
        print("=" * 80)
        print(f"START TEST VEHICUL: {self.vehicle_id}")
        print("=" * 80)
        print(f"Timp planificat: {planned_time:.1f} ore")
        print(f"Distanță estimată: {planned_time * avg_speed:.0f} km")
        print(f"Viteza medie: {avg_speed:.0f} km/h")
        print(f"Interval verificare: {check_interval:.1f} ore")
        print(f"Sisteme testate: {len(self.systems)}")
        print()
        
        test_active = True
        elapsed_time = 0.0
        stop_reason = "Timp planificat atins"
        
        while test_active and elapsed_time < planned_time:
            # Simulare interval
            interval = min(check_interval, planned_time - elapsed_time)
            
            print(f"⏱ Ora {elapsed_time:.1f} - {elapsed_time + interval:.1f} " +
                  f"| Km: {elapsed_time * avg_speed:.0f} - {(elapsed_time + interval) * avg_speed:.0f}")
            
            # Testare fiecare sistem
            interval_defects = []
            for sys_type, system in self.systems.items():
                defects = system.simulate_operation(interval, avg_speed)
                
                if defects:
                    for defect in defects:
                        interval_defects.append(defect)
                        print(f"  ⚠️  {sys_type.value}: {defect.severity.value} - {defect.description}")
                        print(f"      Component: {defect.component} | Cod: {defect.error_code}")
                        print(f"      Timp: {defect.time_occurred:.2f}h | Km: {defect.odometer:.0f}")
                        
                        # Verificare oprire pe defect critic
                        if stop_on_critical and defect.severity == DefectSeverity.CRITICAL:
                            print(f"\n❌ TEST OPRIT: Defect critic detectat în {sys_type.value}")
                            stop_reason = f"Defect critic în {sys_type.value}"
                            test_active = False
                            break
                
                # Verificare stare sistem
                if system.condition < stop_condition_threshold:
                    print(f"\n❌ TEST OPRIT: {sys_type.value} sub pragul minim " +
                          f"({system.condition:.1f}% < {stop_condition_threshold}%)")
                    stop_reason = f"Stare critică {sys_type.value}"
                    test_active = False
                    break
                
                print(f"  ✓ {sys_type.value}: Stare {system.condition:.1f}%")
            
            if not test_active:
                break
            
            elapsed_time += interval
            self.total_test_time = elapsed_time
            self.total_distance = elapsed_time * avg_speed
            
            # Log
            self.test_log.append({
                'time': elapsed_time,
                'distance': self.total_distance,
                'defects': interval_defects,
                'system_conditions': {k: v.condition for k, v in self.systems.items()}
            })
            
            print()
        
        # Finalizare test
        print("=" * 80)
        print("TEST FINALIZAT")
        print("=" * 80)
        print(f"Motiv oprire: {stop_reason}")
        print(f"Timp total: {self.total_test_time:.2f} ore")
        print(f"Distanță totală: {self.total_distance:.0f} km")
        
        # Sumarizare rezultate
        results = self._generate_results(stop_reason)
        
        return results
    
    def _generate_results(self, stop_reason: str) -> Dict:
        """Generează raportul final de rezultate"""
        
        all_defects = []
        system_results = {}
        
        for sys_type, system in self.systems.items():
            all_defects.extend(system.defects)
            
            passed = (system.condition >= 20 and 
                     not any(d.severity == DefectSeverity.CRITICAL for d in system.defects))
            
            system_results[sys_type] = TestResult(
                system=sys_type,
                test_duration=self.total_test_time,
                distance_covered=self.total_distance,
                defects_found=system.defects,
                passed=passed,
                final_condition=system.condition
            )
        
        # Statistici defecte
        defect_stats = {
            'total': len(all_defects),
            'by_severity': {
                severity: len([d for d in all_defects if d.severity == severity])
                for severity in DefectSeverity
            },
            'by_system': {
                sys_type: len(system.defects)
                for sys_type, system in self.systems.items()
            }
        }
        
        results = {
            'vehicle_id': self.vehicle_id,
            'test_duration': self.total_test_time,
            'distance_covered': self.total_distance,
            'stop_reason': stop_reason,
            'system_results': system_results,
            'all_defects': all_defects,
            'defect_statistics': defect_stats,
            'overall_passed': all(r.passed for r in system_results.values())
        }
        
        return results
    
    def generate_report(self, results: Dict, output_dir: str = 'vehicle_test_results'):
        """Generează raport complet cu vizualizări"""
        
        print("\n" + "=" * 80)
        print("GENERARE RAPORT")
        print("=" * 80)
        
        Path(output_dir).mkdir(exist_ok=True)
        
        # 1. Raport text
        self._generate_text_report(results, output_dir)
        
        # 2. Raport Excel
        self._generate_excel_report(results, output_dir)
        
        # 3. Vizualizări
        self._generate_visualizations(results, output_dir)
        
        # 4. JSON cu date raw
        self._save_json_data(results, output_dir)
        
        print(f"\n✓ Raport complet salvat în: {output_dir}/")
    
    def _generate_text_report(self, results: Dict, output_dir: str):
        """Generează raport text"""
        
        report_path = f"{output_dir}/raport_test_{self.vehicle_id}.txt"
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write(f"RAPORT TEST VEHICUL: {results['vehicle_id']}\n")
            f.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 80 + "\n\n")
            
            # Rezumat
            f.write("REZUMAT TEST\n")
            f.write("-" * 80 + "\n")
            f.write(f"Durată test: {results['test_duration']:.2f} ore\n")
            f.write(f"Distanță parcursă: {results['distance_covered']:.0f} km\n")
            f.write(f"Motiv oprire: {results['stop_reason']}\n")
            f.write(f"Status general: {'✓ TRECUT' if results['overall_passed'] else '❌ EȘUAT'}\n\n")
            
            # Statistici defecte
            stats = results['defect_statistics']
            f.write("STATISTICI DEFECTE\n")
            f.write("-" * 80 + "\n")
            f.write(f"Total defecte: {stats['total']}\n\n")
            
            f.write("Pe severitate:\n")
            for severity, count in stats['by_severity'].items():
                f.write(f"  {severity.value:15s}: {count:3d}\n")
            
            f.write("\nPe sistem:\n")
            for sys_type, count in stats['by_system'].items():
                f.write(f"  {sys_type.value:25s}: {count:3d}\n")
            
            # Rezultate per sistem
            f.write("\n\n")
            f.write("REZULTATE PER SISTEM\n")
            f.write("=" * 80 + "\n")
            
            for sys_type, result in results['system_results'].items():
                f.write(f"\n{sys_type.value}\n")
                f.write("-" * 80 + "\n")
                f.write(f"Status: {'✓ Trecut' if result.passed else '❌ Eșuat'}\n")
                f.write(f"Stare finală: {result.final_condition:.1f}%\n")
                f.write(f"Defecte găsite: {len(result.defects_found)}\n")
                
                if result.defects_found:
                    f.write("\nDefecte detaliate:\n")
                    for i, defect in enumerate(result.defects_found, 1):
                        f.write(f"\n  {i}. {defect.severity.value} - {defect.component}\n")
                        f.write(f"     Descriere: {defect.description}\n")
                        f.write(f"     Cod eroare: {defect.error_code}\n")
                        f.write(f"     Apărut la: {defect.time_occurred:.2f}h / {defect.odometer:.0f}km\n")
        
        print(f"✓ Raport text: {report_path}")
    
    def _generate_excel_report(self, results: Dict, output_dir: str):
        """Generează raport Excel"""
        
        excel_path = f"{output_dir}/raport_test_{self.vehicle_id}.xlsx"
        
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Sheet 1: Rezumat
            summary_data = {
                'Parametru': [
                    'Vehicul ID',
                    'Durată test (ore)',
                    'Distanță (km)',
                    'Motiv oprire',
                    'Total defecte',
                    'Defecte minore',
                    'Defecte moderate',
                    'Defecte majore',
                    'Defecte critice',
                    'Status general'
                ],
                'Valoare': [
                    results['vehicle_id'],
                    f"{results['test_duration']:.2f}",
                    f"{results['distance_covered']:.0f}",
                    results['stop_reason'],
                    results['defect_statistics']['total'],
                    results['defect_statistics']['by_severity'][DefectSeverity.MINOR],
                    results['defect_statistics']['by_severity'][DefectSeverity.MODERATE],
                    results['defect_statistics']['by_severity'][DefectSeverity.MAJOR],
                    results['defect_statistics']['by_severity'][DefectSeverity.CRITICAL],
                    'TRECUT' if results['overall_passed'] else 'EȘUAT'
                ]
            }
            pd.DataFrame(summary_data).to_excel(writer, sheet_name='Rezumat', index=False)
            
            # Sheet 2: Rezultate sisteme
            system_data = []
            for sys_type, result in results['system_results'].items():
                system_data.append({
                    'Sistem': sys_type.value,
                    'Status': 'Trecut' if result.passed else 'Eșuat',
                    'Stare Finală (%)': result.final_condition,
                    'Defecte': len(result.defects_found),
                    'Minore': len([d for d in result.defects_found if d.severity == DefectSeverity.MINOR]),
                    'Moderate': len([d for d in result.defects_found if d.severity == DefectSeverity.MODERATE]),
                    'Majore': len([d for d in result.defects_found if d.severity == DefectSeverity.MAJOR]),
                    'Critice': len([d for d in result.defects_found if d.severity == DefectSeverity.CRITICAL])
                })
            pd.DataFrame(system_data).to_excel(writer, sheet_name='Sisteme', index=False)
            
            # Sheet 3: Toate defectele
            if results['all_defects']:
                defect_data = []
                for defect in results['all_defects']:
                    defect_data.append({
                        'Sistem': defect.system.value,
                        'Severitate': defect.severity.value,
                        'Component': defect.component,
                        'Descriere': defect.description,
                        'Cod Eroare': defect.error_code,
                        'Timp (ore)': defect.time_occurred,
                        'Km': defect.odometer
                    })
                pd.DataFrame(defect_data).to_excel(writer, sheet_name='Defecte', index=False)
        
        print(f"✓ Raport Excel: {excel_path}")
    
    def _generate_visualizations(self, results: Dict, output_dir: str):
        """Generează grafice"""
        
        print("\nGenerare grafice...")
        
        # 1. Evoluția stării sistemelor
        fig, ax = plt.subplots(figsize=(14, 8))
        
        for sys_type, system in self.systems.items():
            times = [0]
            conditions = [100]
            
            for log_entry in self.test_log:
                times.append(log_entry['time'])
                conditions.append(log_entry['system_conditions'][sys_type])
            
            ax.plot(times, conditions, marker='o', label=sys_type.value, linewidth=2)
        
        ax.axhline(y=20, color='r', linestyle='--', label='Prag critic', linewidth=2)
        ax.set_xlabel('Timp (ore)', fontsize=12)
        ax.set_ylabel('Stare Sistem (%)', fontsize=12)
        ax.set_title(f'Evoluția Stării Sistemelor - Vehicul {self.vehicle_id}', 
                     fontsize=14, fontweight='bold')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/01_evolutie_stare.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 01_evolutie_stare.png")
        
        # 2. Defecte per sistem
        fig, ax = plt.subplots(figsize=(12, 8))
        
        systems = [sys_type.value for sys_type in results['defect_statistics']['by_system'].keys()]
        counts = list(results['defect_statistics']['by_system'].values())
        
        colors = ['#d73027' if c > 5 else '#fee08b' if c > 2 else '#1a9850' for c in counts]
        bars = ax.barh(systems, counts, color=colors)
        
        ax.set_xlabel('Număr Defecte', fontsize=12)
        ax.set_title('Defecte per Sistem', fontsize=14, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        
        # Adăugare valori
        for bar in bars:
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2, 
                   f'{int(width)}', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/02_defecte_sistem.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 02_defecte_sistem.png")
        
        # 3. Distribuție severitate
        fig, ax = plt.subplots(figsize=(10, 8))
        
        severities = [s.value for s in DefectSeverity]
        counts = [results['defect_statistics']['by_severity'][s] for s in DefectSeverity]
        colors_sev = ['#fee08b', '#fdae61', '#f46d43', '#d73027']
        
        wedges, texts, autotexts = ax.pie(counts, labels=severities, autopct='%1.1f%%',
                                           colors=colors_sev, startangle=90)
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        ax.set_title('Distribuție Defecte după Severitate', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_dir}/03_severitate.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 03_severitate.png")
        
        # 4. Timeline defecte
        if results['all_defects']:
            fig, ax = plt.subplots(figsize=(14, 8))
            
            severity_colors = {
                DefectSeverity.MINOR: '#fee08b',
                DefectSeverity.MODERATE: '#fdae61',
                DefectSeverity.MAJOR: '#f46d43',
                DefectSeverity.CRITICAL: '#d73027'
            }
            
            for defect in results['all_defects']:
                ax.scatter(defect.time_occurred, defect.system.value,
                          c=severity_colors[defect.severity],
                          s=200, alpha=0.7, edgecolors='black')
            
            ax.set_xlabel('Timp (ore)', fontsize=12)
            ax.set_ylabel('Sistem', fontsize=12)
            ax.set_title('Timeline Apariție Defecte', fontsize=14, fontweight='bold')
            ax.grid(alpha=0.3)
            
            # Legendă
            from matplotlib.patches import Patch
            legend_elements = [Patch(facecolor=color, label=sev.value)
                             for sev, color in severity_colors.items()]
            ax.legend(handles=legend_elements, loc='upper right')
            
            plt.tight_layout()
            plt.savefig(f'{output_dir}/04_timeline.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("  ✓ 04_timeline.png")
    
    def _save_json_data(self, results: Dict, output_dir: str):
        """Salvează date în format JSON"""
        
        # Convertire obiecte în dict pentru JSON
        json_data = {
            'vehicle_id': results['vehicle_id'],
            'test_duration': results['test_duration'],
            'distance_covered': results['distance_covered'],
            'stop_reason': results['stop_reason'],
            'overall_passed': results['overall_passed'],
            'defect_statistics': {
                'total': results['defect_statistics']['total'],
                'by_severity': {k.value: v for k, v in results['defect_statistics']['by_severity'].items()},
                'by_system': {k.value: v for k, v in results['defect_statistics']['by_system'].items()}
            },
            'defects': [
                {
                    'system': d.system.value,
                    'severity': d.severity.value,
                    'component': d.component,
                    'description': d.description,
                    'error_code': d.error_code,
                    'time_occurred': d.time_occurred,
                    'odometer': d.odometer
                }
                for d in results['all_defects']
            ]
        }
        
        json_path = f"{output_dir}/date_test_{self.vehicle_id}.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        
        print(f"✓ Date JSON: {json_path}")


def main():
    """Funcția principală - exemplu de utilizare"""
    
    print("\n" + "=" * 80)
    print("SISTEM DE TESTARE AUTOVEHICULE")
    print("Simulare teste până la defect sau timp planificat")
    print("=" * 80)
    
    # Configurare test
    VEHICLE_ID = "TEST-MD-2024-001"
    PLANNED_TIME = 100.0  # ore
    AVG_SPEED = 80.0  # km/h
    CHECK_INTERVAL = 5.0  # verificare la fiecare 5 ore
    
    # Sisteme de testat (sau None pentru toate)
    systems_to_test = [
        SystemType.ENGINE,
        SystemType.TRANSMISSION,
        SystemType.BRAKES,
        SystemType.SUSPENSION,
        SystemType.ELECTRICAL,
        SystemType.COOLING
    ]
    
    # Creare tester
    tester = VehicleTester(VEHICLE_ID, systems=systems_to_test)
    
    # Rulare test
    results = tester.run_test(
        planned_time=PLANNED_TIME,
        check_interval=CHECK_INTERVAL,
        avg_speed=AVG_SPEED,
        stop_on_critical=True,
        stop_condition_threshold=20.0
    )
    
    # Generare raport complet
    tester.generate_report(results)
    
    print("\n" + "=" * 80)
    print("✓ TEST COMPLET FINALIZAT")
    print("=" * 80)
    print(f"\nVezi rezultatele în directorul: vehicle_test_results/")


if __name__ == "__main__":
    main ()


SISTEM DE TESTARE AUTOVEHICULE
Simulare teste până la defect sau timp planificat
START TEST VEHICUL: TEST-MD-2024-001
Timp planificat: 100.0 ore
Distanță estimată: 8000 km
Viteza medie: 80 km/h
Interval verificare: 5.0 ore
Sisteme testate: 6

⏱ Ora 0.0 - 5.0 | Km: 0 - 400
  ✓ Motor: Stare 100.0%
  ✓ Transmisie: Stare 100.0%
  ✓ Frâne: Stare 100.0%
  ✓ Suspensie: Stare 100.0%
  ✓ Sistem Electric: Stare 100.0%
  ✓ Răcire: Stare 100.0%

⏱ Ora 5.0 - 10.0 | Km: 400 - 800
  ✓ Motor: Stare 99.9%
  ✓ Transmisie: Stare 100.0%
  ✓ Frâne: Stare 99.9%
  ✓ Suspensie: Stare 99.9%
  ✓ Sistem Electric: Stare 100.0%
  ✓ Răcire: Stare 100.0%

⏱ Ora 10.0 - 15.0 | Km: 800 - 1200
  ✓ Motor: Stare 99.9%
  ✓ Transmisie: Stare 99.9%
  ✓ Frâne: Stare 99.9%
  ✓ Suspensie: Stare 99.9%
  ✓ Sistem Electric: Stare 100.0%
  ✓ Răcire: Stare 99.9%

⏱ Ora 15.0 - 20.0 | Km: 1200 - 1600
  ✓ Motor: Stare 99.9%
  ✓ Transmisie: Stare 99.9%
  ✓ Frâne: Stare 99.8%
  ✓ Suspensie: Stare 99.9%
  ✓ Sistem Electric: Stare 99.9%
 

Testarea pe bază de timp (Time-Based Reliability Demonstration)

Se stabilește:

o durată de test (ex: 1.000 h),

un nivel de încredere (90%, 95%),

un MTBF țintă.

Se testează până la atingerea timpului planificat sau până la apariția defectelor.